Compare CBOW and Skip gram

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.linalg

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [ ]:
# CBOW is a window view; we are trying to infer the word in the middle.
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

raw_text= """Long Short-Term Memory (LSTM) is a recurrent neural network (RNN) architecture that has been designed
to address the vanishing and exploding gradient problems of conventional RNNs. Unlike feedforward neural networks,
RNNs have cyclic connections making them powerful for modeling sequences. 
They have been successfully used for sequence labeling and sequence prediction tasks,
such as handwriting recognition, language modeling, phonetic labeling of acoustic frames. However, in contrast to the deep neural
networks, the use of RNNs in speech recognition has been limited to phone recognition in small scale tasks. 
In this paper, we present novel LSTM based RNN architectures which make more effective
use of model parameters to train acoustic models for large vocabulary speech recognition. 
We train and compare LSTM, RNN and DNN models at various numbers of parameters and configurations.
We show that LSTM models converge quickly and give state of the art speech recognition performance for relatively small sized models.""".split()

# By deriving a set from "raw_text", we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

# Basic Tokenizer
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(len(raw_text))
print(vocab_size)

152
106


In [ ]:
print(vocab)

{'However,', 'designed', 'present', 'of', 'limited', 'networks,', 'such', 'train', 'this', 'state', 'them', 'RNNs', 'make', 'successfully', 'converge', 'architecture', '(LSTM)', 'give', 'Unlike', 'making', 'recurrent', 'the', 'architectures', 'in', 'vanishing', 'powerful', 'conventional', 'has', 'contrast', 'use', 'modeling', 'tasks,', 'which', 'compare', 'In', 'RNNs.', 'deep', 'connections', 'have', 'recognition,', 'at', '(RNN)', 'for', 'labeling', 'cyclic', 'quickly', 'scale', 'as', 'LSTM,', 'modeling,', 'Short-Term', 'effective', 'recognition.', 'to', 'feedforward', 'exploding', 'model', 'we', 'more', 'large', 'that', 'They', 'models', 'used', 'recognition', 'phone', 'Long', 'based', 'novel', 'paper,', 'language', 'phonetic', 'various', 'configurations.', 'address', 'vocabulary', 'problems', 'is', 'a', 'performance', 'Memory', 'acoustic', 'numbers', 'show', 'been', 'art', 'speech', 'small', 'DNN', 'parameters', 'sized', 'sequence', 'handwriting', 'We', 'neural', 'RNN', 'sequences.',

In [ ]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())

In [ ]:
# Now lets create a "dataset"
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = []
    for j in range(CONTEXT_SIZE, 0, -1):
        context.append(raw_text[i - j])

    for j in range(1, CONTEXT_SIZE + 1):
        context.append(raw_text[i + j])
        
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


[(['Long', 'Short-Term', '(LSTM)', 'is'], 'Memory'), (['Short-Term', 'Memory', 'is', 'a'], '(LSTM)'), (['Memory', '(LSTM)', 'a', 'recurrent'], 'is'), (['(LSTM)', 'is', 'recurrent', 'neural'], 'a'), (['is', 'a', 'neural', 'network'], 'recurrent')]


### Create the CBOW Model 

In [ ]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embed_dim, context, hidden_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(context*embed_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size),
            nn.LogSoftmax(dim = -1)
        )
        
    def forward(self, inputs):
#         print(inputs.shape)
#         print(inputs)
        out = self.embedding(inputs)
#         print(out.shape)
        out = out.view(1, -1)
#         print(out.shape)
        out = self.linear(out)
#         print(out.shape)
        return out
    
    # This is what we are actually interested on
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out


In [ ]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

example_tensor = torch.randint(0, VOCAB_SIZE, [BATCH_SIZE, FULL_CONTEXT_SIZE])
print(example_tensor)

tensor([[56, 93, 61, 99],
        [13, 41, 62, 63],
        [41, 53, 41,  9],
        [99, 95, 96, 56],
        [57, 67, 47, 90],
        [18, 28, 49, 92]])


In [ ]:
CBOW_embedding = nn.Embedding(VOCAB_SIZE, EMBEDD_DIM)
print(example_tensor.shape)
example_result = CBOW_embedding(example_tensor)
# Now we have a representation of the words in a vector of EMBEDD_DIM Dimensions
print(example_result.shape)
# example_result = torch.flatten(example_result, start_dim=1)
example_result = example_result.view(BATCH_SIZE, -1)
print(example_result.shape)

torch.Size([6, 4])
torch.Size([6, 4, 10])
torch.Size([6, 40])


In [ ]:
print("input shape: ", EMBEDD_DIM * FULL_CONTEXT_SIZE)
print("output shape: ", HIDDEN_SIZE)
CBOW_hidden = nn.Linear(EMBEDD_DIM * FULL_CONTEXT_SIZE, HIDDEN_SIZE)
CBOW_hidden_relu = nn.ReLU()
example_result = CBOW_hidden(example_result)
example_result = CBOW_hidden_relu(example_result)
print(example_result.shape)

input shape:  40
output shape:  256
torch.Size([6, 256])


In [ ]:
CBOW_output = nn.Linear(HIDDEN_SIZE, VOCAB_SIZE)
CBOW_output_soft = nn.LogSoftmax(dim = -1)
example_result = CBOW_output(example_result)
example_result = CBOW_output_soft(example_result)
print(example_result.shape)

torch.Size([6, 106])


In [ ]:
print(example_result[0].argmax(-1))
print(key_list[val_list.index(example_result[0].argmax(-1))])
print(example_result[0])
print(example_result[1].argmax(-1))
print(key_list[val_list.index(example_result[1].argmax(-1))])
print(example_result[1])
# print(example_result[2].argmax(-1))
# print(example_result[3].argmax(-1))
# print(example_result[4].argmax(-1))

tensor(19)
making
tensor([-4.3891, -4.3643, -4.3086, -4.6528, -4.9453, -4.8049, -4.3962, -5.1550,
        -4.5120, -4.8235, -4.6538, -4.5381, -4.5556, -4.9118, -4.6445, -4.6424,
        -4.7857, -4.8407, -4.5767, -4.0854, -4.7217, -4.5164, -4.5236, -4.5620,
        -4.9682, -4.7532, -4.5537, -4.7135, -4.5360, -4.6594, -4.7924, -4.9868,
        -4.5479, -4.2876, -4.9942, -4.5309, -4.9476, -4.7529, -4.5620, -4.8405,
        -4.5981, -4.6998, -4.4383, -4.7512, -4.6034, -4.8913, -4.6188, -4.6298,
        -5.1006, -4.6063, -4.4227, -4.4656, -4.5061, -4.8747, -4.6328, -4.6264,
        -5.1929, -5.1265, -4.8233, -4.7178, -4.7838, -4.9191, -4.8638, -4.4865,
        -4.7804, -4.8521, -4.4034, -4.4078, -4.8800, -5.1198, -4.8030, -4.7955,
        -4.7833, -4.3207, -4.6004, -4.5293, -4.8787, -4.5434, -4.4170, -4.9157,
        -4.4425, -4.5870, -4.7609, -4.9018, -4.7502, -5.3236, -4.6879, -4.5138,
        -4.4256, -4.3972, -4.2254, -5.0288, -4.6666, -4.6111, -4.5695, -4.4622,
        -4.8179, -4.98

In [ ]:
# Simple helper method to transform the context to the expected int vector - tensor

def make_context_vector(context, word_to_ix, debug=False):
    if debug:
      print(context)
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix, debug=True)

['Long', 'Short-Term', '(LSTM)', 'is']


tensor([66, 50, 16, 77])

### Training the model

In [ ]:
def train(model, epochs, data, optimizer, loss_fn):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for context, target in data:

            # Prepare inputs and targets 
            context_idxs = make_context_vector(context, word_to_ix)
            context_idxs = context_idxs.to(device)
            target_id = make_context_vector([target], word_to_ix)
            target_id = target_id.to(device)

            # Do not accumulate 
            model.zero_grad()

            # Step 3. Run the forward pass
            log_probs = model(context_idxs)
    #         break

            # Step 4. Compute your loss function.
            loss = loss_fn(log_probs, target_id)

    #         loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
    

In [ ]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

loss_function = nn.NLLLoss() # Because we are using Log_softmax
model = CBOW(vocab_size, EMBEDD_DIM, FULL_CONTEXT_SIZE, HIDDEN_SIZE)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

losses = train(model, 100, data, optimizer, loss_function)
model.eval()

print(losses)  # The loss decreased every iteration over the training data!

[695.375720500946, 688.5787932872772, 681.8794357776642, 675.2739353179932, 668.756386756897, 662.3222517967224, 655.9675674438477, 649.688939332962, 643.4841105937958, 637.3494062423706, 631.2798478603363, 625.2733011245728, 619.3261457681656, 613.4336252212524, 607.5907979011536, 601.7943716049194, 596.0388697385788, 590.3212624788284, 584.6361153125763, 578.9821462631226, 573.352801322937, 567.7468277215958, 562.1607278585434, 556.5900280475616, 551.0327762961388, 545.4840953946114, 539.9431265592575, 534.4101410508156, 528.8793861865997, 523.3506255149841, 517.821415066719, 512.2893575429916, 506.75455063581467, 501.2145540714264, 495.6701162457466, 490.11576825380325, 484.5538958311081, 478.98509126901627, 473.40774965286255, 467.81996488571167, 462.22564589977264, 456.61989945173264, 451.00534227490425, 445.3843162357807, 439.75501012802124, 434.1200684309006, 428.4774900972843, 422.83063235878944, 417.18168818950653, 411.53053960204124, 405.87596291303635, 400.2199435830116, 394

In [ ]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())

In [ ]:
def similarity_cbow(word_1, word_2):
    
    # test word similarity
    print(word_1)
    print(word_2)
    w1_id = torch.tensor(word_to_ix[word_1], dtype=torch.long)
    w2_id = torch.tensor(word_to_ix[word_2], dtype=torch.long)
    w1_id = w1_id.to(device)
    w2_id = w2_id.to(device)
    
    word_1_vec = model.get_word_vector(w1_id)
    word_2_vec = model.get_word_vector(w2_id)
    
    # The norm of a vector (1D-matrix) is the square root of the sum of all the squared values within the vector.
    print(math.sqrt(torch.square(word_1_vec).sum()))    
    print(torch.linalg.norm(word_1_vec))
    print(torch.linalg.norm(word_2_vec))
    print(word_1_vec.dot(word_2_vec))
    
    word_distance = torch.linalg.norm(word_1_vec - word_2_vec)
    print("Distance between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_distance))
    word_similarity = (word_1_vec.dot(word_2_vec) / (torch.linalg.norm(word_1_vec) * torch.linalg.norm(word_2_vec)))
    print("Similarity between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_similarity))


In [ ]:
similarity_cbow("neural", "network")

neural
network
2.2017660075359773
tensor(2.2018, grad_fn=<LinalgVectorNormBackward0>)
tensor(2.2804, grad_fn=<LinalgVectorNormBackward0>)
tensor(-1.7161, grad_fn=<DotBackward0>)
Distance between 'neural' & 'network' : 3.6715
Similarity between 'neural' & 'network' : -0.3418


## Predict middle words

In [ ]:
def predict_middle_word(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = model(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(prev_words, out_word, post_words)

In [ ]:
predict_middle_word("a recurrent", "network is")
predict_middle_word("LSTM is", "recurrent neural")

['a', 'recurrent'] neural ['network', 'is']
['LSTM', 'is'] RNN ['recurrent', 'neural']


## Part of exercises

### Skip Gram

In [ ]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.vocab_size = n_vocab
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        out = scores.view(1, -1)
        log_ps = self.log_softmax(out)
        return log_ps

    # This is what we are actually interested on
    def get_word_vector_skipgram(self, word):
        out = self.embed(word)
        return out

In [ ]:
losses = []
embedding_dim=8 # you can change, if you want

model_skipgram = SkipGram(len(vocab), embedding_dim).to(device)
model_skipgram.eval
criterion = nn.NLLLoss()
optimizer = optim.Adam(model_skipgram.parameters(), lr=0.01)
losses    = train(model_skipgram, 100, data, optimizer, loss_function)

print('Losses: ',losses)
print('Mean Loss: ',np.mean(losses))

Losses:  [934.4411134719849, 832.4653315544128, 760.8177740573883, 695.162679195404, 635.362813949585, 581.601461648941, 534.4822797775269, 494.42684960365295, 461.3823392391205, 435.03235626220703, 414.7642281651497, 399.47274738550186, 387.89311718940735, 379.0238458663225, 372.12974831461906, 366.67082266509533, 362.26031409204006, 358.62915617227554, 355.5897488743067, 353.0092786550522, 350.79144521057606, 348.8669810593128, 347.18498080968857, 345.73189271241426, 344.5988119766116, 344.4588977638632, 346.403887339402, 346.1447341234889, 343.0034300001098, 341.4397175348131, 339.89411589415977, 339.6674537071958, 338.7554791662842, 338.821722750552, 338.1743257017806, 338.4705478390679, 337.7932696690186, 338.05406628713536, 336.881374867633, 336.3124788124114, 334.653426236473, 334.2143654967658, 333.5952400956303, 333.29431931627914, 332.91857713647187, 332.7213660441339, 332.47340037161484, 332.47809925861657, 332.73681113217026, 335.5902278647991, 340.7925998603605, 335.136333

In [ ]:
model_skipgram.eval

<bound method Module.eval of SkipGram(
  (embed): Embedding(106, 8)
  (output): Linear(in_features=8, out_features=106, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)>

## Similarities between words and skim gram

In [ ]:
def similarity_skipgram(word_1, word_2):
    
    # test word similarity
    print(word_1)
    print(word_2)
    w1_id = torch.tensor(word_to_ix[word_1], dtype=torch.long)
    w2_id = torch.tensor(word_to_ix[word_2], dtype=torch.long)
    w1_id = w1_id.to(device)
    w2_id = w2_id.to(device)
    
    word_1_vec = model_skipgram.get_word_vector_skipgram(w1_id)
    word_2_vec = model_skipgram.get_word_vector_skipgram(w2_id)
    
    # The norm of a vector (1D-matrix) is the square root of the sum of all the squared values within the vector.
    print(math.sqrt(torch.square(word_1_vec).sum()))    
    print(torch.linalg.norm(word_1_vec))
    print(torch.linalg.norm(word_2_vec))
    print(word_1_vec.dot(word_2_vec))
    
    word_distance = torch.linalg.norm(word_1_vec - word_2_vec)
    print("Distance between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_distance))
    word_similarity = (word_1_vec.dot(word_2_vec) / (torch.linalg.norm(word_1_vec) * torch.linalg.norm(word_2_vec)))
    print("Similarity between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_similarity))

In [ ]:
similarity_skipgram("neural", "network")

neural
network
4.055859581918756
tensor(4.0559, grad_fn=<LinalgVectorNormBackward0>)
tensor(5.9439, grad_fn=<LinalgVectorNormBackward0>)
tensor(-0.9146, grad_fn=<DotBackward0>)
Distance between 'neural' & 'network' : 7.3219
Similarity between 'neural' & 'network' : -0.0379
